In [8]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [80]:
from typing import Optional

from sqlmodel import Session, select
from pydantic import BaseModel

from app.models import Service
from app import database
from app.main import app

<IPython.core.display.Javascript object>

In [69]:
class TokenBase(BaseModel):
    type: str


class UserToken(TokenBase):
    user: str


class ServiceToken(TokenBase):
    service: str
    origin: str
    user: str
    service_from_db: Optional[Service]

    @property
    def service_exists_in_database(self):
        service = self.fetch_service_from_db()
        return service is not None

    def fetch_service_from_db(self):
        with Session(database.engine) as session:
            # protect against validating an access_token from a deleted service
            service = session.exec(
                select(Service).where(Service.name == self.name)
            ).first()
        self.service_from_db = service
        return service


class DeploymentToken(TokenBase):
    deployment: int

<IPython.core.display.Javascript object>

In [73]:
payloads = [
    {"type": "user", "user": "foobar"},
    {
        "type": "service",
        "service": "fastdeploy",
        "origin": "GitHub",
        "user": "foo",
    },
    {"type": "deployment", "deployment": 10},
]


def payload_to_token(payload):
    type_to_token = {
        "user": UserToken,
        "service": ServiceToken,
        "deployment": DeploymentToken,
    }
    return type_to_token[payload.get("type")].parse_obj(payload)


for payload in payloads:
    print(payload_to_token(payload))

type='user' user='foobar'
type='service' service='fastdeploy' origin='GitHub' user='foo' service_from_db=None
type='deployment' deployment=10


<IPython.core.display.Javascript object>

In [67]:
st = ServiceToken.parse_obj(
    {
        "type": "service",
        "name": "fastdeploy",
        "origin": "GitHub",
        "user": "foo",
    }
)

asdfajsdlfkajsdlfkasjdflaskdjflwq
kwargs:  {'type': 'service', 'name': 'fastdeploy', 'origin': 'GitHub', 'user': 'foo'}
service:  name='fastdeploy' id=1


<IPython.core.display.Javascript object>

In [68]:
print(st.service)

name='fastdeploy' id=1


<IPython.core.display.Javascript object>

In [64]:
with Session(database.engine) as session:
    # protect against validating an access_token from a deleted service
    services = session.exec(
        select(Service)
    ).all()



<IPython.core.display.Javascript object>

In [65]:
services

[Service(name='fastdeploy', id=1)]

<IPython.core.display.Javascript object>

In [77]:
from time import time
from datetime import datetime

<IPython.core.display.Javascript object>

In [78]:
time()

1635660587.693299

<IPython.core.display.Javascript object>

In [82]:
app.url_path_for("steps", 3)

TypeError: Starlette.url_path_for() takes 2 positional arguments but 3 were given

<IPython.core.display.Javascript object>

In [84]:
app.url_path_for("steps", step_id=1)

'/steps/1'

<IPython.core.display.Javascript object>